In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
df = spark.\
    read.\
    format("csv").\
    option("header", True).\
    option("inferSchema", True).\
    load('/Volumes/superstore_databricks_dbt/source/raw/superstore/')

df_schema = df.schema

df = spark.readStream.\
    format("csv").\
    option("header", True).\
    schema(df_schema).\
    load("/Volumes/superstore_databricks_dbt/source/raw/superstore/").\
    toDF(*[c.strip().replace(" ", "_") for c in df_schema.fieldNames()]).\
    withColumn("date_added", current_timestamp()).\
    withColumn("date_updated", current_timestamp())
df.writeStream\
    .outputMode("append")\
    .format("delta")\
    .option("checkpointLocation", "/Volumes/superstore_databricks_dbt/bronze/checkpoint")\
    .trigger(once=True)\
    .toTable("superstore_databricks_dbt.bronze.superstore")

In [0]:
%sql
select * from superstore_databricks_dbt.bronze.superstore limit 10